In [275]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import csv
import sys, os
sys.path.append('~/dsi/capstones/cap_3/')

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
import pyspark as ps

import scipy as stats
import re
import dateparser
import datetime
import math

import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols

pd.set_option('display.max_columns', None)


In [276]:
df_org = pd.read_csv('../data/gsm.csv')

/Users/cv/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (75,76,77,79,80,81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [277]:
df_org.head(10)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,launch_status,body_dimensions,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,memory_internal,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,battery,battery_talk_time,misc_colors,misc_price,network_3g_bands,network_4g_bands,network,network_speed,platform_gpu,main_camera_features,sound,features,selfie_camera_features,battery_stand-by,tests_performance,tests_display,tests_camera,tests_loudspeaker,tests_audio_quality,tests_battery_life,display_protection,battery_charging,misc_models,comms_nfc,misc_sar_eu,battery_music_play,selfie_camera,display,misc_sar,features_browser,body_keyboard,main_camera_dual,comms_infrared_port,selfie_camera_dual,body_build,camera,memory_phonebook,memory_call_records,features_messaging,features_games,features_java,memory,sound_alert_types,features_clock,features_alarm,features_languages,main_camera_triple,selfie_camera_v1,main_camera,main_camera_quad,selfie_camera_triple,main_camera_v1,main_camera_dual_or_triple,main_camera_five
0,Acer,Chromebook Tab 10,No cellular connectivity,NaN,No,No,"2018, March","Available. Released 2018, July",238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in),544.3 g (1.20 lb),No,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB","32 GB, 4 GB RAM",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,No,"3.1, Type-C 1.0 reversible connector",Accelerometer,Non-removable Li-Po 4500 mAh battery (34 Wh),Up to 9 h (multimedia),"Black, Blue",About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,"2016, August","Available. Released 2016, October",191.7 x 101 x 9.4 mm (7.55 x 3.98 x 0.37 in),260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","16/32 GB, 2 GB RAM","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",Non-removable Li-Ion 3400 mAh battery (12.92 Wh),Up to 9 h (multimedia),Black,About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)","HSPA 42.2/11.5 Mbps, LTE Cat4 150/50 Mbps",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,"2016, August","Available. Released 2016, December",153.8 x 75.6 x 8.5 mm (6.06 x 2.98 x 0.33 in),169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","32 GB, 3 GB RAM","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4

In [278]:
df = df_org.copy()

## DATE CONVERSION

In [279]:

# Parsing variables
years = range(1994, 2017)

months = ['January', 'February', 'March', 'April', 'May', 'June', 
         'July', 'August', 'September', 'October', 'November', 'December']

months_short = ['Jan-', 'Feb-', 'Mar-', 'Apr-', 'May-', 'Jun-', 
                'Jul-', 'Aug-', 'Sep-', 'Oct-', 'Nov-', 'Dec-']

months_num = np.linspace(0.0, 1.0, num=12)

months_int = range(1, 12)

months_dict = {
    'January':1, 
    'February':2, 
    'March':3, 
    'April':4, 
    'May':5, 
    'June':6, 
    'July':7, 
    'August':8, 
    'September':9, 
    'October':10, 
    'November':11, 
    'December':12
}

quarters = ['Q1', '1Q', 'Q2', '2Q', 'Q3', '3Q', 'Q4', '4Q']

quarters_dict = {
  'Q1': 'February',
  '1Q': 'February',
  'Q2': 'May',
  '2Q': 'May',
  '3Q': 'August',
  'Q3': 'August',
  '4Q': 'November',
  'Q4': 'November'
}

In [280]:
def parse_date(date_str):
    
    # Removes preceding and proceding spaces
    date_str = date_str.strip()
    
    # Changes strings with "?" to "remove"
    if '?' in date_str:
        date_str = 'remove'
    
    # Removes errant release dates
    if '.' in date_str:
        split_str = date_str.split('.')
        date_str = split_str[0]
    
    # Changes Quarter to Middle Month of Quarter
    for key, val in quarters_dict.items():
        if key in date_str:
            split_2 = date_str.split()
            split_2[split_2.index(key)] = val
            date_str = " ".join(split_2)

    # Converts Year Only to Year + ' July'
    if len(date_str) < 5:
        date_str += ' July'    
    
    # If date is formated in short form i.e. "Feb-01"
    m_index = 0
    for m in months_short:
        if m in date_str:
            split_3 = date_str.split('-')
            split_3[0] = months[m_index]
            split_3[1] = f'20{split_3[1]}'
            date_str = " ".join(split_3)
        m_index += 1
       
    # Date Parsing str using module dataparser    
    date_tm = dateparser.parse(date_str)
    if isinstance(date_tm, datetime.date):
        date_tm = date_tm - datetime.timedelta(days=date_tm.day-1)
        
        
    return date_tm



# parse_date('2009  February. Released 2009  April')
# parse_date('2009  Q1. Released 2009  April')
# parse_date('2012  Q3')
# parse_date('1Q 2003')
# parse_date('2003')
# parse_date('Feb-01')


In [281]:
# Remove Date-less Objects
def refactor_df(df, col='launch_announced'):
    step_df = df[df['launch_announced'].notna()]
    
    step_df['launch_announced'] = step_df['launch_announced'].apply(parse_date)
    
    step_df['launch_announced'] = step_df['launch_announced'].dt.to_period('M')
    
    return step_df

In [282]:
df = refactor_df(df)

## Drop Features and/or Devices

In [283]:
df.columns

Index(['oem', 'model', 'network_technology', 'network_2g_bands',
       'network_gprs', 'network_edge', 'launch_announced', 'launch_status',
       'body_dimensions', 'body_weight', 'body_sim', 'body', 'display_type',
       'display_size', 'display_resolution', 'platform_os', 'platform_chipset',
       'platform_cpu', 'memory_card_slot', 'memory_internal',
       'main_camera_single', 'main_camera_video', 'selfie_camera_single',
       'selfie_camera_video', 'sound_loudspeaker', 'sound_3.5mm_jack',
       'comms_wlan', 'comms_bluetooth', 'comms_gps', 'comms_radio',
       'comms_usb', 'features_sensors', 'battery', 'battery_talk_time',
       'misc_colors', 'misc_price', 'network_3g_bands', 'network_4g_bands',
       'network', 'network_speed', 'platform_gpu', 'main_camera_features',
       'sound', 'features', 'selfie_camera_features', 'battery_stand-by',
       'tests_performance', 'tests_display', 'tests_camera',
       'tests_loudspeaker', 'tests_audio_quality', 'tests_battery_lif

In [284]:
useless_drop_words = [
    'launch_status',
    'misc_colors',
    'misc_models',
    'misc_sar_eu',
    'misc_sar',
    'memory',
    'sound_alert_types',
    'features_clock',
    'features_alarm',
    'features_languages',
    'selfie_camera_v1',
    'main_camera',
    'main_camera_quad',
    'selfie_camera_triple',
    'main_camera_v1',
    'main_camera_five'
    
]

apple_drop_words = [
    'body_keyboard',
    'memory_phonebook',
    'memory_call_records',
    'features_messaging',
    'features_games',
    'features_java'
    
]

analysis_drop_words = [
    'body_dimensions',
    'memory_internal',
    'battery',
    'battery_talk_time',
    'network_speed',
    'battery_stand-by',
    'tests_performance',
    'tests_display',
    'tests_camera',
    'tests_loudspeaker',
    'tests_audio_quality',
    'tests_battery_life',
    'battery_music_play',
    'selfie_camera',
    'display',
    'features_browser',
    'comms_infrared_port',
    'selfie_camera_dual'
    
]

In [285]:
# Drop Useless Features
df.drop(columns=useless_drop_words, inplace=True)

In [286]:
# Drop Useless Features
df.drop(columns=apple_drop_words, inplace=True)

In [287]:
# Drop Useless Features
df.drop(columns=analysis_drop_words, inplace=True)

In [288]:
df.head(5)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple
0,Acer,Chromebook Tab 10,No cellular connectivity,NaN,No,No,2018-03,544.3 g (1.20 lb),No,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,No,"3.1, Type-C 1.0 reversible connector",Accelerometer,About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Acer,Iconia Tab 10 A3-A40,No cellular connectivity,NaN,No,No,2016-04,-,No,NaN,"IPS LCD capacitive touchscreen, 16M colors","10.1 inches, 295.8 cm (~68.4% screen-to-body r...","1920 x 1200 pixels, 16:10 ratio (~224 ppi dens...",Android 6.0 (Marshmallow),Mediatek MT8163A,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",5 MP,1080p@30fps,2 MP,NaN,"Yes, with stereo speakers (4 speakers)",Yes,"Wi-Fi 802.11 a/b/g/n/ac, dual-band, hotspot","4.0, A2DP",NaN,No,microUSB 2.0,Accelerometer,About 230 EUR,Na

In [289]:
# df.groupby('oem').count().model > 25

## Cast NaN/Null

In [315]:

make_nan = ['No', 'N/A', 'No cellular connectivity', '-']

def cast_nan(step_df, make_nan):

    df_obj = step_df.select_dtypes(['object'])

    step_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

    step_df.replace(to_replace=make_nan, value=np.nan, inplace=True)
    
    return step_df

In [316]:
df = cast_nan(df, make_nan)

In [318]:
df.head(5)

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch,accelerometer,gyro,heart rate,fingerprint,compass,proximity,barometer,spo2,iris scanner,gesture,tempurature,altimeter,infrared face recognition
0,Acer,Chromebook Tab 10,NaN,NaN,NaN,NaN,2018-03,544.3 g (1.20 lb),NaN,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,NaN,"3.1, Type-C 1.0 reversible connector",Accelerometer,About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Acer,Iconia Ta

## PARSE PHONE, WATCH, TABLET

In [290]:
tablet_words = ['tab', 'tablet', 'pad', 'book']
watch_words = ['watch', 'gear', 'fit', 'band']

In [291]:
# df_samsung = df.loc[df['oem'] == 'Samsung', :]
# df_samsung

In [292]:
df_acer = df.loc[df['oem'] == 'Acer', :]

In [293]:
def parse_type(model_string, words_list):

    for check_word in words_list:
#         print(check_word)
        if check_word in model_string.lower():
            return True
        else:
            return False


In [294]:
df['is_tablet'] = df['model'].apply(lambda x: parse_type(x, tablet_words))

df['is_watch'] = df['model'].apply(lambda x: parse_type(x, watch_words))

In [295]:
df

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch
0,Acer,Chromebook Tab 10,No cellular connectivity,NaN,No,No,2018-03,544.3 g (1.20 lb),No,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,No,"3.1, Type-C 1.0 reversible connector",Accelerometer,About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,Acer,Iconia Tab 10 A3-A40,No cellular connectivity,NaN,No,No,2016-04,-,No,NaN,"IPS LCD capacitive touchscreen, 16M colors","10.1 inches, 295.8 cm (~68.4% screen-to-body r...","1920 x 1200 pixels, 16:10 ratio (~224 ppi dens...",Android 6.0 (Marshmallow),Mediatek MT8163A,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",5 MP,1080p@30fps,2 MP,NaN,"Yes, with stereo speakers (4 speakers)",Yes,"Wi-Fi 802.11 a/b/g/n/ac, dual-band, hotsp

## PRICE - FEATURE

In [296]:
# df[''] = data['result'].map(lambda x: x.lstrip('+-').rstrip('aAbBcC'))

In [297]:
# df_acer['misc_price']

In [298]:
def clean_price(price_value): 

#     price_words = ['About']
#     currencies = ['USD', 'EUR', 'INR']

    if type(price_value)!=str:
        if math.isnan(price_value):
            return np.nan
        
    elif price_value == '-':
        return np.nan
    
    elif price_value == 'About BTC 0.15/ETH 4.78(crypto curr)':
        return 800.0
    
    else:
        price_value = price_value.split(' ')

        if price_value[2] == 'EUR':
            return float(price_value[1]) / 1.2
        elif price_value[2] == 'INR':
            return float(price_value[1]) * 60
        elif price_value[2] == 'USD':
            return float(price_value[1])


In [299]:
# np.where(df_acer['misc_price'].values)


In [300]:
df['misc_price'].apply(lambda x: clean_price(x))

0       275.000000
1       141.666667
2       208.333333
3       100.000000
4       191.666667
5       191.666667
6              NaN
7       166.666667
8        91.666667
9       291.666667
10      183.333333
11       91.666667
12       75.000000
13             NaN
14      166.666667
15             NaN
16      125.000000
17             NaN
18             NaN
19      208.333333
20      150.000000
21      166.666667
22      108.333333
23       75.000000
24       66.666667
25      108.333333
26      166.666667
27      125.000000
28      216.666667
29      150.000000
           ...    
8950           NaN
8951           NaN
8952           NaN
8953           NaN
8954           NaN
8955           NaN
8956           NaN
8957           NaN
8958     50.000000
8959           NaN
8960    150.000000
8961           NaN
8962           NaN
8963           NaN
8964           NaN
8965           NaN
8966           NaN
8967           NaN
8968           NaN
8969           NaN
8970           NaN
8971        

## ENCODE SENSOR FEATURES

In [301]:
    relevant_sensors = ['accelerometer', 'gyro', 'heart rate', 'fingerprint', 'compass',
                        'proximity', 'barometer', 'spo2', 'iris scanner', 'gesture', 
                        'tempurature', 'altimeter', 'infrared face recognition'
                       ]

In [302]:
def extract_features(features_values, data_frame, regex_1, relevant_features = None,
                     regex_2 = None, col = None):
    
    if relevant_features:
        for val in relevant_features:
            df[val] = False
    
    features_list = re.split(str(regex_1), str(features_values))

    for feat in features_list:
        for val in relevant_features:
            if val in feat.lower():
                data_frame[str(val)] = True
        
    return features_values
            
            
#     if regex_1:
#         if 'yes' in features_values:
#             return np.nan
    

In [307]:
df['features_sensors'].apply(
    lambda x: extract_features(
                                x, 
                                data_frame=df, 
                                regex_1=', ', 
                                relevant_features=relevant_sensors
    )
)


0                                           Accelerometer
1                                Accelerometer, proximity
2       Fingerprint (front-mounted), accelerometer, pr...
3                                Accelerometer, proximity
4                                           Accelerometer
5                       Accelerometer, proximity, compass
6                 Accelerometer, gyro, proximity, compass
7                       Accelerometer, proximity, compass
8                       Accelerometer, proximity, compass
9                                                     Yes
10                Accelerometer, gyro, proximity, compass
11                               Accelerometer, proximity
12                               Accelerometer, proximity
13                      Accelerometer, proximity, compass
14                      Accelerometer, proximity, compass
15                               Accelerometer, proximity
16                               Accelerometer, proximity
17            

In [306]:
df

,oem,model,network_technology,network_2g_bands,network_gprs,network_edge,launch_announced,body_weight,body_sim,body,display_type,display_size,display_resolution,platform_os,platform_chipset,platform_cpu,memory_card_slot,main_camera_single,main_camera_video,selfie_camera_single,selfie_camera_video,sound_loudspeaker,sound_3.5mm_jack,comms_wlan,comms_bluetooth,comms_gps,comms_radio,comms_usb,features_sensors,misc_price,network_3g_bands,network_4g_bands,network,platform_gpu,main_camera_features,sound,features,selfie_camera_features,display_protection,battery_charging,comms_nfc,main_camera_dual,body_build,camera,main_camera_triple,main_camera_dual_or_triple,is_tablet,is_watch,accelerometer,gyro,heart rate,fingerprint,compass,proximity,barometer,spo2,iris scanner,gesture,tempurature,altimeter,infrared face recognition
0,Acer,Chromebook Tab 10,No cellular connectivity,NaN,No,No,2018-03,544.3 g (1.20 lb),No,Stylus,"IPS LCD capacitive touchscreen, 16M colors","9.7 inches, 291.4 cm (~71.0% screen-to-body ra...","1536 x 2048 pixels, 4:3 ratio (~264 ppi density)",Chrome OS,Rockchip RK3399,Hexa-core (4x Cortex-A53 & 2x Cortex-A72),"microSD, up to 128 GB",5 MP,720p,2 MP,NaN,"Yes, with stereo speakers",Yes,"Wi-Fi 802.11 b/g/n/ac, dual-band, WiFi Direct","4.1, A2DP, LE",NaN,No,"3.1, Type-C 1.0 reversible connector",Accelerometer,About 330 EUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Acer,Iconia Talk S,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2,Yes,Yes,2016-08,260 g (9.17 oz),Dual SIM (Micro-SIM/Nano-SIM),NaN,"IPS LCD capacitive touchscreen, 16M colors","7.0 inches, 135.1 cm (~69.8% screen-to-body ra...","720 x 1280 pixels, 16:9 ratio (~210 ppi density)",Android 6.0 (Marshmallow),Mediatek MT8735,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)","13 MP, AF",1080p@30fps,2 MP,720p,Yes,Yes,"Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS, GLONASS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 170 EUR,HSDPA 850 / 1900 / 2100,"LTE band 1(2100), 3(1800), 7(2600), 8(900), 20...","LTE band 3(1800), 7(2600), 8(900), 28(700)",Mali-T720MP2,"HDR, panorama",DTS HD sound,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,Acer,Liquid Z6 Plus,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,169 g (5.96 oz),Single SIM (Micro-SIM) or Hybrid Dual SIM (Mic...,NaN,"IPS LCD capacitive touchscreen, 16M colors","5.5 inches, 83.4 cm (~71.7% screen-to-body ratio)","1080 x 1920 pixels, 16:9 ratio (~401 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6753 (28 nm),Octa-core 1.3 GHz Cortex-A53,"microSD, up to 256 GB (uses shared SIM slot)","13 MP, AF",1080p@30fps,5 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, Wi-Fi Direct, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Fingerprint (front-mounted), accelerometer, pr...",About 250 EUR,HSDPA,LTE,NaN,Mali-T720MP3,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Acer,Liquid Z6,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (d...,Yes,Yes,2016-08,126 g (4.44 oz),"Single SIM (Micro-SIM) or Dual SIM (Micro-SIM,...",NaN,"IPS LCD capacitive touchscreen, 16M colors","5.0 inches, 68.9 cm (~65.3% screen-to-body ratio)","720 x 1280 pixels, 16:9 ratio (~294 ppi density)",Android 6.0 (Marshmallow),Mediatek MT6737 (28 nm),Quad-core 1.25 GHz Cortex-A53,"microSD, up to 256 GB","8 MP, AF",Yes,2 MP,NaN,Yes,Yes,"Wi-Fi 802.11 b/g/n, hotspot","4.0, A2DP","Yes, with A-GPS",FM radio,microUSB 2.0,"Accelerometer, proximity",About 120 EUR,HSDPA,LTE,NaN,Mali-T720MP2,"LED flash, HDR, panorama",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False

In [308]:
a = 0
for vals in df['features_sensors']:
    print(a)
    extract_features(vals, df, ', ')
    a+=1

0


TypeError: 'NoneType' object is not iterable

## VIDEO - FEATURE


In [309]:
df_acer['main_camera_video']

0                         720p
1                  1080p@30fps
2                  1080p@30fps
3                          Yes
4                  1080p@30fps
5                  1080p@30fps
6     2160p@30fps, 1080p@30fps
7                  1080p@30fps
8                  1080p@30fps
9                  1080p@30fps
10    2160p@30fps, 1080p@30fps
11                         Yes
12                         Yes
13                 1080p@30fps
14                         Yes
15                         Yes
16                         Yes
17                         Yes
18                         Yes
19                 1080p@30fps
20                 1080p@30fps
21                 1080p@30fps
22     1080p@30fps, 720p@60fps
23                  720p@30fps
24                         Yes
25              Yes, dual shot
26                 1080p@30fps
27                 1080p@30fps
28                 1080p@30fps
29                 1080p@30fps
                ...           
64                        720p
65      

In [310]:
def clean_video(video_camera_value):
    
    if type(video_camera_value)!=str:
        if math.isnan(video_camera_value):
            return np.nan
    
    if '@' in video_camera_value:
        return 'Yes'
    
    elif 'x' in video_camera_value:
        video_camera_value = re.split('x|@', video_camera_value)
        
#         video_camera_value = video_camera_value.split('x')
#         [x+1 if x >= 45 else x+5 for x in l]
# #         video_camera_value = video_camera_value.split('@')
        video_camera_value = int(video_camera_value[0]) * int(video_camera_value[1])
    
    elif video_camera_value != np.nan:
        video_camera_value.replace('w', '')
        video_camera_value = video_camera_value.split('p')[0]
        
        if 'yes' in video_camera_value.lower():
            video_camera_value = 'Yes'
    
    return video_camera_value

In [311]:
# df_acer['main_camera_video'] = df_acer['main_camera_video'].apply(clean_video)


In [312]:
df['main_camera_video'].apply(clean_video)

0         720
1         Yes
2         Yes
3         Yes
4         Yes
5         Yes
6         Yes
7         Yes
8         Yes
9         Yes
10        Yes
11        Yes
12        Yes
13        Yes
14        Yes
15        Yes
16        Yes
17        Yes
18        Yes
19        Yes
20        Yes
21        Yes
22        Yes
23        Yes
24        Yes
25        Yes
26        Yes
27        Yes
28        Yes
29        Yes
        ...  
8950    25344
8951      Yes
8952       No
8953       No
8954       No
8955       No
8956      NaN
8957      Yes
8958      Yes
8959      Yes
8960      Yes
8961      Yes
8962      Yes
8963      Yes
8964      Yes
8965      Yes
8966      Yes
8967      Yes
8968      NaN
8969      NaN
8970      Yes
8971      Yes
8972      Yes
8973      Yes
8974      Yes
8975      Yes
8976      Yes
8977      Yes
8978      Yes
8979      NaN
Name: main_camera_video, Length: 8980, dtype: object

In [313]:
# df.main_camera_video[135]


In [314]:
# a = 0
# for word in df['main_camera_video']:
# #     print(a)
#     print(clean_video(word))
#     a+=1